# 分布式计算 总复习

**Python, PySpark & 数值计算基础**

- lect2：PySpark初始化, map & reduce
- lect3：Python中的函数式编程（思想）
- lect4：函数式编程（MapReduce的实现）& 弹幕
- lect5,6：Numpy, RDD & 数值计算原则 

**机器学习基础**

- lect7：ML-（分布式）矩阵乘法&线性回归
- lect8：岭回归、共轭梯度、logistics（理论）

**优化算法系列**

- lect9: logistics的梯降法、牛顿法实现
- lect10：logistics的L-BFGS实现; 数值稳定算法，缓存，混洗
- lect11：ADMM理论（以LAD，Lasso为例的理论推导）,soft_thresholding
- lect12：LAD与Lasso的ADMM实现（非RDD）, Cholesky分解

## Lect2：PySpark初始化, map & reduce

### 课堂实例：联合国文字的处理

- PySpark配置和启动

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
# 本地模式
spark = SparkSession.builder.master("local[*]").appName("Reading Text").getOrCreate()
sc = spark.sparkContext
# sc.setLogLevel("ERROR")
print(spark)
print(sc)

<SparkContext master=local[*] appName=Reading Text>


- PySpark中文件读取，查看，筛选

    `sc.textFile("address")`

    `file.count()`

    `file.take()`

    `file.filter()`

    `file.map()`


### Slides大纲


#### 1. 并行计算基本概念 
- Amdahl 定律：并行加速程度计算公式
  $$S_{latency}(s)=\frac{1}{(1-p)+p/s}$$
  $S$：整体提升倍数，$s$：可并行部分加速比，$p$：可并行部分所占时间的比例

#### 2. 并行计算 vs 分布式计算

#### 3. Apache Spark简介与安装
- Hadoop 分布式计算框架
- HDFS 分布式文件系统
- MapReduce
- Spark：Hadoop改进




#### 4. Spark运行模式
- 单机模式
- 集群模式

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
# 本地模式
spark = SparkSession.builder.\
    master("local[*]").\
    appName("PySpark RDD").\
    getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")
print(spark)
print(sc)

## Lect3：Python中的函数式编程（思想）

### 课堂实例：迭代器、计算方差、随机抽样、MapReduce

- 迭代器的创建与访问：
  - `iter()`
  - `next(IT)`

- 一次遍历计算方差：

  $$(n-1)S=\sum_i (x_i-\bar{x})^2=\sum_i x_i^2-n\bar{x}^2$$

- 随机抽样:
  - `for`循环中一次遍历两个循环
    - `for a,b in zip(A,B):`
  - 遍历的同时获取索引
    - ```python
      for i, (w, v) in enumerate(zip(wvec, vvec)):
      print(f"i is {i}, w is {w}, v is {v}")
      ```

- Reduce:
  - syntax:
    ```python
        import functools
        functools.reduce(FUNCTION,ITER,OPTIONAL)
    ```
    `OPTIONAL` 是一个可选选项，用来设定reduce的初始数值（例如在累乘时要设置初值为1）
        

- Filter:
  - syntax:
    - `filter(IS_FUNCTION,ITER)`

      其中`IS_FUNCTION`是一个需要返回`TRUE/FALSE`的函数

- Map:
  - syntax:
    - `map(FUNCTION,ITER)`

- islice （按照指定长度截断迭代器）:
  - syntax:
    - ```python
        import itertools
        itertools.islice(ITER,NUM)
        ```

- `lambda`函数

### slides 讲解

- 函数式编程
  - 概念、特点
  - 核心思想
  - 迭代器：YOLO！
  - 迭代器全部取出为列表：`lst=list(it)`

## Lect4：函数式编程（MapReduce的实现）& 弹幕

- 样本方差计算
    ```python
    num,sum,sq_sum = data.map(lambda x: (1,x,x*x)).reduce(lambda x,y: (x[0]+y[0],x[1]+y[1],x[2]+y[2]))
    mean = sum / num
    sample_var = (sq_sum+num*mean*mean-2*mean*sum)/(num-1)
    print(sample_var)
    ```

## Lect5,6：Numpy, RDD & 数值计算原则

### 5.1 Numpy

#### 基础操作

- `np.array([1,2,3],[3,4,5])` 定义数列（矩阵）

- `np.linspace(start= 1 , stop=5 , num= 12 )`  生成连续数列

- `np.arange(12)` 生成[0,1,...,12]的array

- `np.reshape(VEC, (3,4))` / `vec.reshape(3,4)`

- `np.ones((3,2))` !不是生成单位矩阵，而是全1矩阵

- `np.zeros((2,3))` 零矩阵

- `np.eyes(5)` 只需要输入一个数字，即可生成对应单位阵

- `np.diag([1,3,5])` 对角矩阵

- `vec[-1]` 表示倒数第1个元素

- `vec[start:end]` 左闭右开

- `mat[:,:2]` 表示全部行，前两列（注意python中下标从1开始）

- `np.random.uniform(low = , high = , size = )` 均匀分布

- `np.random.normal(loc = 2 , scale = 3, size = (2,5))` 正态分布 *loc为均值，scale为标准差（非方差）*

- `np.log()` `np.exp()` 对数与指数

- `np.sum(MAT, axis = 0)` 矩阵的求和，其中`axis=0` 表示对每个列求一个和

- `np.mean(MAT, axis = 1)` 矩阵的平均，其中`axis = 1` 表示对每个行求一个平均


>汇总可以按行或者按列进行，这由`axis`参数决定。（联想到矩阵元素下标$a_{ij}$是先行后列，故这里`axis`是0行1列）0表示运算时第一个维度（行）在变化，1表示运算时第二个维度（列）在变化。再次提醒，Python中以0表示第一个元素！


#### 线性代数

- `MAT.transpose()` 转置

- `MAT_A.dot(MAT_B)` / `np.matmul(MAT_A,MAT_B)` 矩阵乘法

- `np.linalg.inv(MAT)` 矩阵的逆(不推荐)

- `np.linalg.solve(B,d)` 解线性方程组 (返回$B^{-1}d$)

- `evals, evecs = np.linalg.eigh(MAT)` 求特征值，特征向量



### 5.2 RDD

PySpark 初始化


In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
# 本地模式
spark = SparkSession.builder.\
    master("local[*]").\
    appName("PySpark RDD").\
    getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")
print(spark)
print(sc)

#### RDD的创建

- `DAT = sc.parallelize(VEC)` 将一个数据(i.e. VEC)转化为rdd对象(i.e. DAT)

- `DAT.collect()` 收集所有内容

- `DAT.count()` 返回的实质为RDD的迭代次数

- `DAT.take()` / `DAT.first()` 取RDD前几个next的内容



#### RDD 对.txt文件的处理

- `file = sc.textFile("ADDRESS")` 读入文本文件

- `print(*(file.take(5)), sep="\n")` 对前五行的打印

- 将string类型数据转换成numpy数据：


In [ ]:
# str => np.array
def str_to_vec(line):
    # 分割字符串
    str_vec = line.split("\t")
    # 让 Numpy 进行类型转换
    return np.array(str_vec, dtype=float)
DAT = file.map(str_to_vec)    

#### RDD的分区

- `file.getNumPartitions()` 获得自动分区数

- `file.repartition(NUM)` 自定义分区数

- 对于切分完的分区，我们希望把一个分区中的数据按照一个矩阵来理解，故：




In [ ]:
 # str => np.array
def str_to_vec(line):
    # 分割字符串
    str_vec = line.split("\t")
    # 让 Numpy 进行类型转换
    return np.array(str_vec, dtype=float)
DAT = file.map(str_to_vec)    
 # Iter[str] => Iter[matrix]
def part_to_mat(iterator):
    # Iter[str] => Iter[np.array]
    iter_arr = map(str_to_vec, iterator)


    # Iter[np.array] => list(np.array)
    dat = list(iter_arr) #dat是由向量（nparray）作为元素组成的列表（list）

    # 有的分区可能是空分区
    # list(np.array) => matrix
    if len(dat) < 1:  # Test zero iterator
        mat = np.array([])
    else:
        mat = np.vstack(dat) 

    # matrix => Iter[matrix]
    yield mat # yield可以认为是return[mat]，返回的不是mat本身，而是包含这个的容器

dat_p10 = file_p10.mapPartitions(part_to_mat)

dat_p10_nonempty = dat_p10.filter(lambda x: x.shape[0] > 0)


### 5.3 数值计算原则

#### 原则1：矩阵相乘，小维度优先

- 经验法则：对于更一般的矩阵乘法 $A_{m\times n}B_{n\times p}C_{p\times r}$，如果 $n\approx p$ 且 $m>r$，则优先计算 $BC$，反之优先计算 $AB$。

- 对于一个$A_{n\times p }\times B_{p\times q}$的矩阵乘法，时间复杂度为$O(npq)$

#### 原则2：尽量避免显式矩阵求逆

常见矩阵运算复杂度整理：

（假设$A,B : n\times n, b :n\times1$）

矩阵乘法：
- $AB:O(n^3)$
- $Ab:O(n^2)$

矩阵的逆：
- $A^{-1}:O(n^3)$
- $A^{-1}b:O(n^3)$ （但从实证方面看比求逆效率更高）

矩阵的其他运算：
- $|A|, eign(A) :O(n^3) $
- 特别的：上/下三角矩阵的行列式$O(n)$ （行列式为对角线的乘积）
- $||A||_p^2 : O(n^2)$
- $ A+b1^T : O(n^2)$ 相当于将$b$广播



#### 原则3：利用矩阵的特殊结构

- 对于对角矩阵，在存储时应当以向量存储对角线元素即可，事实上在参与计算中也可以通过广播机制直接以向量参与运算：
  - 假设$D=diag(d_1,...,d_n)$为对角线元素，$A$为正常矩阵，则$DA$相当于对第i列乘以$d_i$，$AD$相当于对第i行乘以$d_i$

#### 原则4：尽可能将显示循环转化为矩阵计算

- 虽然理论复杂度相似，但实证上，考虑通信成本等，矩阵运算更快

In [ ]:
# 1. 初始化pyspark

import findspark
findspark.init("/Users/xinby/Library/Spark")

from pyspark.sql import SparkSession
# 本地模式
spark = SparkSession.builder.\
    master("local[*]").\
    appName("PySpark RDD").\
    getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")
print(spark)
print(sc)

# 2. 生成数据

import numpy as np
np.set_printoptions(linewidth=100)

np.random.seed(123)
n = 100
p = 5
mat = np.random.normal(size=(n, p))
np.savetxt("mat_np.txt", mat, fmt="%f", delimiter="\t")

# 3. 读取数据至RDD并分区

file = sc.textFile("mat_np.txt")
file_p10 = file.repartition(10)
# str => np.array
def str_to_vec(line):
    # 分割字符串
    str_vec = line.split("\t")
    # 将每一个元素从字符串变成数值型
    num_vec = map(lambda s: float(s), str_vec)
    # 创建 Numpy 向量
    return np.fromiter(num_vec, dtype=float)

# Iter[str] => Iter[matrix]
def part_to_mat(iterator):
    # Iter[str] => Iter[np.array]
    iter_arr = map(str_to_vec, iterator)

    # Iter[np.array] => list(np.array)
    dat = list(iter_arr)

    # list(np.array) => matrix
    if len(dat) < 1:  # Test zero iterator
        mat = np.array([])
    else:
        mat = np.vstack(dat)

    # matrix => Iter[matrix]
    yield mat

dat = file_p10.mapPartitions(part_to_mat).filter(lambda x: x.shape[0] > 0)

## Lect 7：ML-（分布式）矩阵乘法&线性回归（OLS）

### 7.1 矩阵乘法（RDD）

#### 0. *准备工作*

#### 1. 矩阵乘法$Xv$

**原理**

假设：$ X \in \R^{n\times p}, v \in \R^{p}$

将$X$按照行进行分块（包含所有列，但行不一定是一行），记为：$X=[X_1;...;X_m]^T$, $X_i \in \R^{n_i\times p}$

故$Xv=[X_1v;...;X_mv]^T$

In [ ]:
np.random.seed(123)
v = np.random.uniform(size=p)

# 这里不是reduce是因为不是要把最后分块进行加和，而是想要最后的拼接
res_part = dat.map(lambda x: x.dot(v)).collect() 
print(res_part,type(res_part))
# 这个结果里，一个array就是前面的一个分块
np.concatenate(res_part)

#### 2. 矩阵乘法 $X'X$

**原理**

同理进行分块，则最终的结果为：$X'X=X_1'X_1+\dots+X_m'X_m$ (由于这里的假定$X\in\R^{n\times p}, n>>p$，故最终的大小为$p\times p$，是可以存储在内存中的)

In [ ]:
res = dat.map(lambda x: x.T.dot(x)).reduce(lambda x, y: x + y) 
# x代表了前面的累积结果，y代表了最新一项

#### 3. 矩阵乘法 $X'v$

**原理**

这里设定$X,v$的行数相同，故可以同时进行拆分(在具体操作时认为是在一个大矩阵中)，$X'v = X_1'v_1+...+X_m'v_m$

In [ ]:
X = mat[:, :-1]
v = mat[:, -1]
def Xitv(part):
    '''
    在定义这个map函数的时候，由于整体是一个大矩阵，但是在内部需要将X和v区分开来
    '''
    Xi = part[:, :-1]
    vi = part[:, -1]
    return Xi.transpose().dot(vi)

res = dat.map(Xitv).reduce(lambda x, y: x + y)

### 7.2 线性回归（OLS） （RDD）

**理想OLS的理论模型与假设：**

$$y = \beta_0 + \beta'x +\epsilon$$

其中 $X\in \R^{n\times p}$，这里要求$n>>p$

$$\hat\beta = (X'X)^{-1}X'y$$

- 若要包含截距项，则$X$的第一列为1

**计算过程**
1. 首先计算$X'X,X'y$
2. 然后计算$(X'X)^{-1}X'y$
   
事实上，由矩阵分块原理，可以只计算一次：$X'[X;y]$


#### 实例：RDD实现 （hw4）

##### 参数估计

In [ ]:
xt_xy = data_partition_nonempty.\
    map(lambda x: np.hstack((np.ones((np.shape(x)[0],1)),x))).\
    map(lambda x: x[:,:-1].transpose().dot(x) ).\
    reduce (lambda x,y: x+y)

xt_x = xt_xy[:,:-1]
xt_y = xt_xy[:,-1]

hat_beta = np.linalg.solve(xt_x,xt_y)
print(hat_beta)

##### R_sq

相关公式：
$$SSR = \sum (y_i-\hat y_i)^2 = ||Y-X\hat\beta||_2$$
$$SST = \sum (y_i - \bar y)^2 = \sum y_i^2+n\bar y^2-2n\bar y \sum y_i $$
$$ R^2 = 1 - SSR/SST$$

假设：
1. 已知$\hat\beta, [X,Y]$
2. 数据经过mapPartition在多个矩阵中存储
   
计算过程：
1. 扩充$X := [1,X]$
2. 计算 $Y-X\hat\beta$，稍后对其进行平方求和
3. 计算 $\sum y_i, \sum y_i^2$
4. reduce，根据上述Rsquare公式进行整合计算

In [ ]:
sum_y, sum_y_sq, ssr, num = data_partition_nonempty.\
    map(lambda x: np.hstack((np.ones((np.shape(x)[0],1)),x))).\
    map(lambda x: (np.sum(x[:,-1]),np.sum(x[:,-1]**2),np.sum(x[:,-1]-x[:,:-1].dot(hat_beta)**2,axis=0),np.shape(x)[0])).\
    reduce(lambda x,y:(x[0]+y[0],x[1]+y[1],x[2]+y[2],x[3]+y[3]) )
print(sum_y,sum_y_sq, ssr, num)
y_bar = sum_y/num
sst = sum_y_sq + n*y_bar**2 -2*num*y_bar*sum_y
R_sq = 1 - ssr / sst
print(f"R^2: {R_sq}")


## Lect8：岭回归、共轭梯度法、logistics（理论）

### 8.2 共轭梯度法

**算法目的**

求解$Ax=b$（只要$A_{n \times n}$是正定的，n步内即求得精确解）

- 时间复杂度最差为$O(n^3)$

*具体算法细节参见codes*

**CG-Python实现**

In [ ]:
def cg(A, b, x0, eps=1e-3, print_progress=False):
    ''' 已知Ax=b中的A与b,求解x
    A: n*n
    b: n*1
    x0: 一个初始迭代内容,可以用np.zeros(shape=m)等初始化
    eps: 精度
    print_progress: 是否打印迭代过程
    '''
    m = b.shape[0]
    # 初始解（注意此处应该复制x0，否则程序退出时会修改x0）
    x = np.copy(x0)
    # 初始残差向量
    r = b - np.dot(A, x)
    # 初始共轭梯度
    p = r

    for k in range(m):
        # 矩阵乘法
        Ap = np.dot(A, p)
        rr = r.dot(r)
        alpha = rr / p.dot(Ap)
        # 更新解
        x += alpha * p
        # 计算新残差向量
        rnew = r - alpha * Ap
        # 测试是否收敛
        norm = np.linalg.norm(rnew)
        if print_progress:
            print(f"Iter {k}, residual norm = {norm}")
        if norm < eps:
            break
        beta = rnew.dot(rnew) / rr
        # 更新共轭梯度
        p = rnew + beta * p
        # 更新残差向量
        r = rnew

    return x

In [ ]:
# 更通用的共轭梯度法：ABSTRACTION!

def cg(Afn, b, x0, eps=1e-3, print_progress=False, **Afn_args):
    ''' 已知Ax=b中的A与b,求解x
    在cg的实际计算中,我们永远只关心A与其他向量的乘法结果,
    而并不关心A本身的数值,因此可以通过函数调用的方式来实现
    Afn: 传入一个函数，该函数应当形如 PROD(X,MATRIX),
         其中大写字母表示可以自定义的名称
         该函数将返回MAT.dot(X)
         其优势在于比如处理对角矩阵时可以通过广播节省计算
    b, x0, eps, print_progress: 与cg函数相同
    Afn_args: 传入Afn的所有其他参数(可以是多个,按序给出即可),譬如MATRIX=A (A就是cg中的那个A)
    '''
    m = b.shape[0]
    # 初始解（注意此处应该复制x0，否则程序退出时会修改x0）
    x = np.copy(x0)
    # 初始残差向量
    r = b - Afn(x, **Afn_args)
    # 初始共轭梯度
    p = r

    for k in range(m):
        # 矩阵乘法
        Ap = Afn(p, **Afn_args)
        rr = r.dot(r)
        alpha = rr / p.dot(Ap)
        # 更新解
        x += alpha * p
        # 计算新残差向量
        rnew = r - alpha * Ap
        # 测试是否收敛
        norm = np.linalg.norm(rnew)
        if print_progress:
            print(f"Iter {k}, residual norm = {norm}")
        if norm < eps:
            break
        beta = rnew.dot(rnew) / rr
        # 更新共轭梯度
        p = rnew + beta * p
        # 更新残差向量
        r = rnew

    return x

def mat_prod(x, mat):
    return mat.dot(x)

def diag_mat_prod(x, diag_elements):
    return diag_elements * x  # 逐元素相乘，非矩阵乘法

sol = cg(diag_mat_prod, b, x0=np.zeros(shape=m), print_progress=True, diag_elements=np.diagonal(A))

### 8.1 岭回归

**理论**

- 当 $n<p$ 时，$X'X$ 不可逆，此时最小二乘**没有唯一解** （事实上是有无数组解使得$\min S_c=0$，即可以完美拟合）【并不是说不存在解！！】
- 此时的OLS并不影响预测性，但是并不影响解释性
- 此时我们可以采用岭回归的方法，其在最小二乘损失函数的基础上加入一个惩罚项 

$$Loss = ||Y-X\beta||_2+\lambda ||\beta||^2$$

$$\hat\beta_\lambda = (X^TX+\lambda I)^{-1}X^TY$$

- 但注意到 $X'X+\lambda I$ 是一个高维的矩阵($p\times p$)，难以直接进行求解。因此我们采用共轭梯度法

**Python 实现**

在岭回归中，我们需要求解 $\hat\beta_\lambda = (X^TX+\lambda I)^{-1}X^TY$

参照cg的形式，也就是对于方程$Ax=b$，求解$x$：其中$A=X^TX+\lambda I, b=X^TY$

观察cg的算法以及前面“更通用的cg”中介绍的，对A的矩阵乘法可以abstracted成一些函数的形式以提高效率。具体而言，也就是在cg中要实现一些形如$Av = (X^TX+\lambda I)v \equiv X^TXv+\lambda v$的乘法

因此重点是思考如何高效计算$X^TXv$:

若将$X$按行分块为$X=(X_1;X_2;...;X_m)'$, 则$X'(Xv) = X_1'X_1v_1+...+X_m'X_mv_m$, 而这便可以分布式实现。

对于rdd中的每一个分块，计算$X_i'(X_iv_i)$，然后将各个分块的结果reduce即可（注意乘法的顺序）

In [ ]:
def xtxv(part, v):
    '''该函数是非分布式,其含义只是单纯输入一个X,v,输出X.TX.v
    part: 从第二列开始是X的数据(第一列为y)
    '''
    x = part[:, 1:]
    return x.transpose().dot(x.dot(v))

def ridge_prod(v, lam, rdd):
    '''计算X'Xv+lam*v,其中X'Xv是分布式计算的
    v,lam: ridge回归的参数
    rdd: 每个rdd为数据的一个分块,形式上相当于一个分块矩阵,对于每个分块矩阵计算一个矩阵乘法,
         最终再用reduce加总起来
    ridge_prod: 返回X'Xv+lam*v
    '''
    first_term = rdd.map(lambda part: xtxv(part, v)).reduce(lambda x, y: x + y)
    second_term = lam * v
    return first_term + second_term

def cg(Afn, b, x0, eps=1e-3, print_progress=False, **Afn_args):
    m = b.shape[0]
    # 初始解（注意此处应该复制x0，否则程序退出时会修改x0）
    x = np.copy(x0)
    # 初始残差向量
    r = b - Afn(x, **Afn_args)
    # 初始共轭梯度
    p = r

    for k in range(m):
        # 矩阵乘法
        Ap = Afn(p, **Afn_args)
        rr = r.dot(r)
        alpha = rr / p.dot(Ap)
        # 更新解
        x += alpha * p
        # 计算新残差向量
        rnew = r - alpha * Ap
        # 测试是否收敛
        norm = np.linalg.norm(rnew)
        if print_progress:
            print(f"Iter {k}, residual norm = {norm}")
        if norm < eps:
            break
        beta = rnew.dot(rnew) / rr
        # 更新共轭梯度
        p = rnew + beta * p
        # 更新残差向量
        r = rnew

    return x
    
b = dat.map(lambda part: part[:, 1:].transpose().dot(part[:, 0])).reduce(lambda x, y: x + y)
lam = 0.01 * n
sol = cg(ridge_prod, b, x0=np.zeros(shape=p), eps=1e-3, print_progress=True, lam=lam, rdd=dat)


### 8.3 Logistics 理论

$$Y|x \sim Bernoulli(\rho(\beta^T x))$$

Sigmoid: $\rho(x) = \frac{1}{1+e^{-x}}$,表示$y=1$的概率

故通过MLE：
$$l = \sum \log P(Y_i =y_i) = \sum[ y_i \log p_i + ( 1 - y_i) \log (1 - p_i)]$$
where $p_i = \rho(x_i' \beta)$

而求解使得$l$最大的$\beta$，需要通过数值解法，这也是后面几节的重点。

In [ ]:
def string_to_vector(line):
    vector = line.split("\t")
    vector = np.append(vector,1.0) #可以通过这个代码添加一行1
    return np.array(vector, dtype=float)

## Lect9: logistics的梯降法、牛顿法实现

**优化目标函数**

$$\argmin \beta = l= \sum[ y_i \log p_i + ( 1 - y_i) \log (1 - p_i)]$$

### 9.1 梯降法

**理论**

$$\beta^{new} = \beta^{old}-\alpha \frac{\partial l}{\partial \beta}|_{\beta=\beta^{old}} =\beta^{old}-\alpha\cdot n^{-1}X'(prob-y)$$
其中$prob=\rho(X\beta^{old})$

梯降法一次迭代的时间复杂度为$O(np)$

**Python实现**

In [ ]:
from scipy.special import expit, logit #expit为sigmoid函数

def compute_obj_grad(part_mat, beta_old):
    '''计算目标函数值和梯度(针对logistics),
    不是梯降法主函数,而是通过调用它来计算梯降法的关键数据
    ----[INPUT]----
    part_mat:rdd,是原始数据的一个分块矩阵,第一列是y,其余列是X
    beta_old:np.array,当前的beta值

    ----[OUTPUT]----
    ni:该分块的样本量
    obj:目标函数值
    grad:梯度(即目标函数在当前beta处的导数值)
    '''
    # 提取 X 和 y
    y = part_mat[:, 0]
    x = part_mat[:, 1:]
    # X * beta
    xb = x.dot(beta_old)
    # rho(X * beta)
    prob = expit(xb)
    # 目标函数：sum(y * log(prob) + (1 - y) * log(1 - prob))
    obj = -np.sum(y * np.log(prob + 1e-8) + (1.0 - y) * np.log(1.0 - prob + 1e-8))
    # 梯度：X'(prob - y)
    grad = x.transpose().dot(prob - y)
    # 该分块的样本量
    ni = x.shape[0]
    return ni, obj, grad

#### 初始化 ####
# 根据数据动态获取维度，p为x变量个数
p = dat.first().shape[1] - 1
# beta 初始化为 0 向量
beta_hat2 = np.zeros(p)
# 记录目标函数值
objvals = []
# 最大迭代次数
maxit = 100
# 步长
step_size = 10.00
# 收敛条件
eps = 1e-5

dat.cache() #缓存机制

#### 迭代 ####
for i in range(maxit):
    # 完整数据的样本量和梯度是各分区的加和（因为导数是具有线性性的）
    n, objfn, grad = dat.map(lambda part: compute_obj_grad(part, beta_hat2)).\
        reduce(lambda x, y: (x[0] + y[0], x[1] + y[1], x[2] + y[2]))
    objfn /= n
    grad /= n
    # 计算新 beta
    beta_new = beta_hat2 - step_size * grad
    # 计算梯度的变化
    grad_norm = np.linalg.norm(grad)
    beta_norm = np.linalg.norm(beta_hat2)
    print(f"Iteration {i}, objfn = {objfn}, grad_norm = {grad_norm}, beta_norm = {beta_norm}")
    objvals.append(objfn)
    # 如果梯度值较小，退出循环
    if grad_norm < eps or grad_norm < eps * beta_norm:
        break
    # 更新 beta
    beta_hat2 = beta_new

### 9.2 牛顿法

**理论**

按照牛顿法general的公式对$l$求导，可以得到针对logistics的牛顿法迭代公式：

$$\beta^{new} = (X'WX)^{-1}X'Wz = (X'WX)^{-1}X'W(X\beta^{old}+W^{-1}(y-prob))$$

其中:

1. $prob=\rho(X\beta^{old})$

2. $W$是一个对角矩阵，$W_{ii}=prob_i(1-prob_i)$

因此又转化为求解$X'WX,X'Wz$，再解线性方程组的问题，并且其中只有$z$是参与迭代的。

*！不要显式存储对角矩阵W，使用广播实现*



**Python实现**

In [ ]:
def compute_stats(part_mat, beta_old):
    # 提取 X 和 y
    y = part_mat[:, 0]
    x = part_mat[:, 1:]
    
    # X * beta
    xb = x.dot(beta_old)

    # rho(X * beta)
    prob = expit(xb)

    # W 的对角线元素
    w = prob * (1.0 - prob) + 1e-6

    # X'W，数组广播操作，避免生成完整的 W
    xtw = x.transpose() * w

    # X'WX
    xtwx = xtw.dot(x)

    # X'Wz
    z = xb + (y - prob) / w

    xtwz = xtw.dot(z)

    # 目标函数：sum(y * log(prob) + (1 - y) * log(1 - prob))
    objfn = -np.sum(y * np.log(prob + 1e-8) + (1.0 - y) * np.log(1.0 - prob + 1e-8))
    return xtwx, xtwz, objfn

# 根据数据动态获取维度，不要使用之前模拟时的变量
p = dat.first().shape[1] - 1
# beta 初始化为 0 向量
beta_hat = np.zeros(p)
# 记录目标函数值
objvals = []
# 最大迭代次数
maxit = 30
# 收敛条件
eps = 1e-6

for i in range(maxit):
    # 完整数据的 X'WX 和 X'Wz 是各分区的加和
    xtwx, xtwz, objfn = dat.map(lambda part: compute_stats(part, beta_hat)).\
        reduce(lambda x, y: (x[0] + y[0], x[1] + y[1], x[2] + y[2]))
    # 计算新 beta
    beta_new = np.linalg.solve(xtwx, xtwz)
    # 计算 beta 的变化
    resid = np.linalg.norm(beta_new - beta_hat)
    print(f"Iteration {i}, objfn = {objfn}, resid = {resid}")
    objvals.append(objfn)
    # 如果 beta 几乎不再变化，退出循环
    if resid < eps:
        break
    # 更新 beta
    beta_hat = beta_new

## Lect10：logistics的L-BFGS实现，数值稳定算法，缓存，混洗

### 10.1 L-BFGS

**Python实现**

`scipy.optimize.minimize`

In [ ]:
from scipy.optimize import minimize
def compute_obj_grad(part_mat, beta_old):
    # 提取 X 和 y
    y = part_mat[:, 0]
    x = part_mat[:, 1:]
    # X * beta
    xb = x.dot(beta_old)
    # rho(X * beta)
    prob = expit(xb)
    # 目标函数：sum(y * log(prob) + (1 - y) * log(1 - prob))
    obj = -np.sum(y * np.log(prob + 1e-8) + (1.0 - y) * np.log(1.0 - prob + 1e-8))
    # 梯度：X'(prob - y)
    grad = x.transpose().dot(prob - y)
    # 该分块的样本量
    ni = x.shape[0]
    return ni, obj, grad

def logistic_obj_grad(beta, *args):
    dat = args[0]
    n, objfn, grad = dat.map(lambda part: compute_obj_grad(part, beta)).\
        reduce(lambda x, y: (x[0] + y[0], x[1] + y[1], x[2] + y[2]))
    objfn /= n
    grad /= n
    return objfn, grad

res = minimize(logistic_obj_grad, #l-bfgs需要提供目标函数和梯度值 \
    beta_init, #要优化变量的初始值 \ 
    args=(dat,), #QUESTION：这里的dat一定是y和x拼接成的吗？函数会自动判别吗？\
    method="L-BFGS-B", \
    jac=True, \
    options={"iprint": 1})


### 10.2 数值稳定算法

#### 问题1：Sigmoid函数

**问题分析**

Sigmoid:
$$\rho(x) = \frac{1}{1+e^{-x}}$$

当$x$很小时，$\exp(x)$很大，造成数值溢出

**问题处理**

解法1：

$ x\ge 0,\quad sigmoid(x)=\frac{1}{1+e^{-x}}$

$ x<0,\quad sigmoid(x)=\frac{e^x}{1+e^x}$

解法2：

直接调用：`scipy.special.expit()`

#### 问题2：Softplus函数

**问题分析**

在计算obj_fn的时候，需要计算：

$$\sum[ y_i \log p_i + ( 1 - y_i) \log (1 - p_i)]$$

即

$\log(\rho(x)) = \log(\frac{1}{1+e^{-x}}) = x-\log(1+e^x)$

$\log(1-\rho(x)) = -\log(1+e^x)$

因此核心就在于计算

$$\operatorname{softplus} = \log(1+e^x)$$

依然是在取指数的时候出现问题，故分类讨论：

$x\ge0, \operatorname{softplus}=x+\log(1+e^{-x})$

$x<0, \operatorname{softplus}=\log(1+e^x)$

因此对数部分可以统一为：

$$\log(1+e^{-|x|})$$

**问题处理**

In [ ]:
def softplus(x):
    log_num = np.log(1+np.exp(-np.abs(x)))
    ans = np.where(x>=0, x+log_num, log_num)
    return ans

### 10.3 RDD缓存

**缓存机制**

在利用 Spark 对 RDD 进行操作时，通常会叠加很多次变换（ map 、 filter 等等）。理论上每次取数据时都要重复整个流程。为了避免重复操作，可以将中间的某些结果进行缓存；即将变换后的计算结果存进内存，下次需要数据时直接从内存调取。

In [ ]:
file = sc.textFile("data/logistic.txt")
dat = file.mapPartitions(part_to_mat).filter(lambda x: x.shape[0] > 0)

##### RDD 缓存 #####
dat.cache()
####################

**增加内存上限**

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
# 本地模式
spark = SparkSession.builder.\
    master("local[*]").\
    config("spark.executor.memory", "2g").\
    config("spark.driver.memory", "2g").\
    appName("Logistic Regression").\
    getOrCreate()
sc = spark.sparkContext
# sc.setLogLevel("ERROR")
print(spark)
print(sc)

### 10.4 RDD混洗

**Shuffling概念**

- `repartition()`会触发shuffle
- 如果想要减少分区数，可以调用`coalesce()`避免shuffle
- 若需要增加分区数，则混洗不可避免，此时要注意问题是否具有排列不变性（行打乱是否有影响）
  - 不依赖顺序的算法：排列不变性，通常是一些汇总计算例如计算方差、回归系数、似然函数等
  - 依赖排序等算法：通常是与观测相关的计算，例如观测的预测值等
- 若无法避免shuffling，但也是排序依赖的，可以用`rdd.zipWithIndex()`来加入索引id，后续对齐

## Lect11：ADMM理论（以LAD，Lasso为例的理论推导,soft_thresholding）

### 11.1 ADMM的理论

**优化目标：**

$$\min_{x,z} f(x)+g(z) \\s.t. Ax+Bz = c$$

其中 $A_{p\times n},x_{n\times 1},B_{p\times m},z_{m\times 1},c_{p\times 1}$ （p可以认为是约束的个数）； $f,g$为凸函数

**ADMM实现过程**


1. 首先引入增广函数:

$$L_\rho(x,z,y) = f(x)+g(z)+y'(Ax+bz-c)\\+(\rho/2) ||Ax+Bz-c||_2^2$$

其中$\rho$为任意给定正数，$y_{p\times 1}$为辅助变量


2. 进行迭代

$$x^{k+1} = \argmin_x L_\rho(x,z^k,y^k) \\ z^{k+1} = \argmin_z L_\rho(x^{k+1},z,y^k) \\ y^{k+1} = y^k + \rho(Ax^{k+1}+Bz^{k+1}-c)$$

3. 设置停止条件

对偶问题残差：$s^{k+1} = \rho A'B(z^{k+1} - z^k)$

原问题残差：$r^{k+1} = Ax^{k+1}+Bz^{k+1}-c$

当两个残差的范数在某次迭代后都小于某个阈值时，停止迭代。


### 11.2 ADMM例1：LAD（理论）

**LAD概念**

- 对异常值稳健
- 损失函数非光滑但为凸函数

**优化目标：**

$$ \min _x \left\| Ax-b \right\| _1 $$

**ADMM对应**

令$f(\cdot)=0,g(\cdot)=\left\| \cdot \right\|， B=-I, c=b$，则

$$\min g(z)\\s.t. Ax-z=b$$

**迭代过程**

$$x^{k+1} := (A'A)^{-1}A'(b+z^k-u^k) \\ z^{k+1} := S_{1/\rho}(Ax^{k+1}-b+u^k) \\ u^{k+1} := u^k + Ax^{k+1} - z^{k+1} - b$$
其中$S_{\kappa}(a)$ - Soft-thresholding operator:

$$S_{\kappa}(a) = \begin{cases} a-\kappa, & a>\kappa \\ 0, & -\kappa\le a\le \kappa \\ a+\kappa, & a<-\kappa \end{cases}$$

一种紧凑的表达是 $S_{\kappa}(a)=\mathrm{sign}(a)\cdot\max\{0,|a|-\kappa\}$。

*上述迭代中的$x$即为所求*

### 11.3 ADMM例2：Lasso（理论）



**Lasso概念**

- 起到变量选择

**优化目标：**

$$ \min _x \frac12 \left\| Ax-b \right\| _2^2 + \lambda \left\| x \right\| _1 $$

**ADMM对应**

令$f(x)=\frac12 \left\| Ax-b \right\| _2^2,g(z)=\lambda \left\| z \right\| _1$,则

$$\min f(x)+g(z)\\s.t. ~ x-z=0$$

**迭代过程**

$$x^{k+1} := (A'A+\rho I)^{-1}(A'b+\rho(z^k-u^k)) \\ z^{k+1} := S_{\lambda/\rho}(x^{k+1}+u^k) \\ u^{k+1} := u^k + x^{k+1} - z^{k+1} $$


## Lect12：LAD与Lasso的ADMM实现（非RDD）,Cholesky分解

### 12.1 Soft-Thresholding 与 Cholesky分解

#### Soft-Thresholding

$$
S_{\kappa}(a)=\begin{cases}
a-\kappa, & a>\kappa\\
0, & |a|\le\kappa\\
a+\kappa, & a<-\kappa
\end{cases},
$$

一种紧凑的表达是 $S_{\kappa}(a)=\mathrm{sign}(a)\cdot\max\{0,|a|-\kappa\}$。

In [ ]:
def soft_thresholding(a, k):
    return np.sign(a) * np.maximum(0.0, np.abs(a) - k)

#### Cholesky Decomposition

**Cholesky分解理论**

当$M$是正定的，定存在分解$M=LL'$，其中$L$是下三角矩阵

如果在一个迭代中，需要循环计算$Mx=b$，则可以将解线性方程组的过程分成两步：1. 首先对$M$进行Cholesky分解 2. 解上三角的线性方程组（解这样的方程组的时间复杂度为$O(n^2)$）。

此外，Cholesky分解的结果（由于$M$矩阵是不变的），因此只需要在迭代外部计算一次即可，在循环中即可快速的通过解上三角的线性方程组来求解，大大减少了时间复杂度。


**Cholesky python实现**

假设要求解: $M^{-1}v$

In [ ]:
from scipy.linalg import cho_factor, cho_solve

# 首先，在循环外可以一次性进行Cholesky分解 
c, lower = cho_factor(M) # 注意cho_factor一次会返回两个值，在应用中直接按照(c,lower)合并成一个元组统一处理即可

# 在循环内，就可以把原先的np.linalg.solve替换成cho_solve
res = cho_solve((c, lower), v)


### 12.2 ADMM-LAD python 实现

In [8]:
# 模拟生成数据
import numpy as np
np.set_printoptions(linewidth=100)
np.random.seed(123)
n = 1000
p = 10
A = np.random.normal(size=(n, p))
xtrue = np.random.normal(size=p)
b = A.dot(xtrue) + np.random.normal(size=n)

# soft-thresholding
from scipy.linalg import cho_factor, cho_solve
def soft_thresholding(a, k):
    return np.sign(a) * np.maximum(0.0, np.abs(a) - k)

# 初始化数值
x = np.zeros(p)
z = np.zeros(n)
u = np.zeros(n)
rho = 1.0 # rho 理论可以是任意正数
max_iter = 10000
tol = 0.001

# Cholesky 分解中间变量
AtA = A.T.dot(A)
c, lower = cho_factor(AtA)

# ADMM迭代
for i in range(max_iter):
    # x 更新
    xnew = cho_solve((c,lower),A.T.dot(b+z-u))
   #xnew = np.linalg.solve(A.T.dot(A), A.T.dot(b + z - u))
    # z 更新
    Axnew = A.dot(xnew)
    znew = soft_thresholding(Axnew - b + u, 1.0 / rho)
    # u 更新
    unew = u + Axnew - znew - b
    # 计算残差大小
    resid_r_norm = np.linalg.norm(unew - u)
    resid_s_norm = rho * np.linalg.norm(A.T.dot(znew - z))
    # 更新 x、z 和 u 的取值
    x = xnew
    z = znew
    u = unew
    # 打印残差信息，判断是否收敛
    if i % 100 == 0:
        continue
        #print(f"Iteration {i}, ||r|| = {resid_r_norm:.6f}, ||s|| = {resid_s_norm:.6f}")
    if resid_r_norm <= tol and resid_s_norm <= tol:
        break

print(x)
print(xtrue)

[-1.19230848 -0.28642899 -0.89053513  2.35251214  0.66217182  0.14198784 -0.43247972 -1.11299057
 -0.01374415 -0.38485577]
[-1.24096967 -0.31294679 -0.84894679  2.37795259  0.65750062  0.21308689 -0.49097031 -1.0815104
  0.00480111 -0.36079657]


### 12.3 ADMM-Lasso python 实现

用 $M\in\mathbb{R}^{n\times p}$ 表示自变量矩阵，$b\in\mathbb{R}^n$ 表示因变量向量 （因此$M,b$都是已知的数据），要估计的回归系数为 $x\in\mathbb{R}^p$。于是 Lasso 的目标函数为 $$\frac{1}{2}\Vert Mx-b\Vert^2+\lambda \Vert x\Vert_1,$$
其迭代公式为
$$
\begin{align*}
x^{k+1} & =(M'M+\rho I)^{-1}(M'b+\rho(z^{k}-u^{k}))\\
z^{k+1} & =S_{\lambda/\rho}(x^{k+1}+u^{k})\\
u^{k+1} & =u^{k}+x^{k+1}-z^{k+1},
\end{align*}
$$

In [13]:
# 模拟数据
import numpy as np
np.set_printoptions(linewidth=100)
np.random.seed(123)
n = 1000
p = 30
nz = 10
M = np.random.normal(size=(n, p))
xtrue = np.random.normal(size=nz)
xtrue = np.concatenate((xtrue, np.zeros(p - nz)))# 真实的 x 只有前10个元素非零，其余均为0
b = M.dot(xtrue) + np.random.normal(size=n)

# soft-thresholding
def soft_thresholding(a, k):
    return np.sign(a) * np.maximum(0.0, np.abs(a) - k)

# 参数及超参数设置
rho = 1.0
lam = 0.01*n
max_iter = 10000
kappa = lam / rho
tol = 0.01

# 迭代元素初始化

MtM  = M.transpose().dot(M)
Mtb = M.transpose().dot(b)
I = np.eye(p)
c, lower = cho_factor(MtM+rho*I)

x = np.zeros(p)
z = np.zeros(p)
u = np.zeros(p)

resid_r = -999
resid_s = -999

for iter in range(max_iter):
    xnew = cho_solve((c,lower),Mtb+rho*(z-u))
    znew = soft_thresholding(xnew+u,kappa)
    unew = u + xnew - znew
    
    resid_r = np.linalg.norm(xnew-znew)
    resid_s = - rho*np.linalg.norm(znew-z)

    x = xnew
    z = znew
    u = unew

    # 打印残差信息，判断是否收敛
    if iter % 200 == 0:
        #print(f"Iteration {iter}, ||r|| = {resid_r:.6f}, ||s|| = {resid_s:.6f}")
        continue
    if resid_r <= tol and resid_s <= tol:
        #print(f"Iteration {iter}, ||r|| = {resid_r:.6f}, ||s|| = {resid_s:.6f}")
        break

print(x)
print(xtrue)


[-1.11993425e+00 -7.75763650e-01  1.81091384e+00  1.75943298e+00  1.31142956e+00 -4.01742626e-01
 -6.19590384e-01 -4.85078523e-01  1.00971039e+00  6.85745010e-01 -1.12150786e-03 -2.96475270e-02
  2.38870164e-02  3.56066046e-03  2.70006717e-02 -1.08262799e-02  2.68398062e-02 -3.71308977e-02
  5.30794866e-03  1.36178951e-02  3.84465382e-02  1.20218888e-02  1.69123155e-02  6.43126083e-02
 -2.09899563e-02  1.60904388e-02  2.59800827e-03 -1.09607367e-02  8.29531241e-03 -1.18662502e-03]
[-1.05417044 -0.78301134  1.82790084  1.7468072   1.3282585  -0.43277314 -0.6686141  -0.47208845
  1.05554064  0.67905585  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]
